# faustWidgets
This notebook relies on [faustWidgets](https://github.com/hrtlacek/faustWidgets) which is installed cloned to `/root/audiolab/faustWidgets` on this system and installed via the `PYTHONPATH` environment variable.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import FAUSTPy
import faustWidgets

In [7]:
sr = 44100
nyq = sr/2

## Synth Example

In [8]:
!cat synthTest.dsp

cat: synthTest.dsp: No such file or directory


In [11]:
faustObject = FAUSTPy.FAUST("synthTest.dsp", sr)

CalledProcessError: Command '['faust', '-lang', 'c', '-single', 'synthTest.dsp']' returned non-zero exit status 1.

The test file is a distorted sine wave oscillator. 

- The oscillators Frequency can be controlled
- a constant DC offset can be added
- the amplifictaion before it's running into a standard $tanh$ distortion can be controlled. 


![block diaram](simpleTest-svg/process.svg)

In [10]:
faustWidgets.getWidgets(faustObject)

Output()

## Effect Example
A Lowpass Filter

![block diaram](effectTest-svg/process.svg)

In [6]:
faustEffect = FAUSTPy.FAUST("effectTest.dsp", sr)

In [7]:
impulse = np.zeros(1000)
impulse[100] = 1
x = impulse
faustWidgets.getWidgets(faustEffect, x = x)

Output()

## Custom Plot Functions

In [8]:
def myPlot(x):
    N = len(x)
    n2 = int(N/2)
    faxis = np.linspace(0,nyq,n2)
    plt.semilogx(faxis, 20*np.log10(abs(np.fft.fft(x.flatten())))[:n2])
    plt.grid()
    plt.show()
    return

In [9]:
faustWidgets.getWidgets(faustObject, plotFun=myPlot, nSamps=4410)

Output()

In [10]:
def myPlot2(x):
    plt.specgram(x.flatten(), Fs=44100)
    plt.grid()
    plt.show()
    return


In [11]:
faustWidgets.getWidgets(faustObject, plotFun=myPlot2, nSamps=44100)

Output()

## Using the Widget values in the Plot
the widget's values are acessible in the plot function if it is constructed to take a second argument. Here, a dictionary with the parameters is passed.
this way, a comparison between a python dsp algorithm and FAUST implementation can be shown.

In [12]:
import scipy.signal as sig

In [13]:
!cat effectTest.dsp

import("stdfaust.lib");
N = 1;
fc = hslider("cutoff", 100, 10, 10000, 0.1);
process = fi.lowpass(N,fc);


In [14]:
def myComparePlot(x, pardict):
#     print(x)
    nyq = 22050
    Wn = pardict['p_cutoff']/nyq
    ba = sig.butter(1,Wn)
    imp = np.zeros_like(x)
    imp[10] = 1
    y = sig.lfilter(ba[0],ba[1],x)
    plt.plot(y,'k', label='python')
    plt.plot(x,'r--', label='faust')
    plt.legend()
    plt.ylim([-0.1,1])
    plt.grid()
    plt.show()
    return


In [15]:
impulse = np.zeros(50)
impulse[10] = 1
x = impulse
faustWidgets.getWidgets(faustEffect, x = x, plotFun=myComparePlot)

Output()